In [1]:
import sqlite3
from rdflib import Graph, Namespace, RDF, RDFS, OWL, URIRef, Literal

In [2]:
def convert_db_to_owl(path):
    conn = sqlite3.connect(path)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    g = Graph()

    owl = Namespace("http://www.w3.org/2002/07/owl#")
    ex = Namespace("http://example.org#")

    k = 0

    try:
        # Iterate over each table
        for table in tables:
            table_name = table[0]
            if k == 2: break
            if table_name == 'synset' or table_name == 'sense':

                k += 1

                print(f"Table: {table_name}")

                cursor.execute(f"PRAGMA table_info({table_name});")
                columns = cursor.fetchall()
                column_names = [col[1] for col in columns]
                print(f"Columns: {column_names}")

                class_uri = ex[table_name]
                g.add((class_uri, RDF.type, owl.Class))

                for col in columns:
                    prop_uri = ex[col[1]]
                    g.add((prop_uri, RDF.type, owl.DatatypeProperty))
                    g.add((prop_uri, RDFS.domain, class_uri))

                cursor.execute(f"SELECT * FROM {table_name};")
                rows = cursor.fetchall()
                for row in rows:
                    instance_uri = ex[f"{table_name}_{row[0]}"]
                    g.add((instance_uri, RDF.type, class_uri))

                    for i, value in enumerate(row):
                        prop_uri = ex[column_names[i]]
                        g.add((instance_uri, prop_uri, Literal(value)))

    finally:
        conn.close()

    g.serialize(destination='ontologies.owl', format='xml')


In [3]:
db_path = r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\ruwordnet-2021.db'
convert_db_to_owl(db_path)

Table: synset
Columns: ['id', 'title', 'definition', 'part_of_speech']
Table: sense
Columns: ['id', 'name', 'lemma', 'synset_id']
